Dou et al. (2020) model solved in parallel

Estimation using Coupled Simulated Annealing, from Xavier et al. (2010).

Agora em Julia versão 1.5.3

This script only solves the model. For a version with unit testing and graphs, refer to "BargainingJulia"



Version to use at HPC - aguia4

Número de simulações = length(pids) * 5

Código para fazer cálculos em duplas

In [1]:
using Distributed

In [2]:
nprocs = 4;

In [3]:
addprocs(nprocs);

In [4]:
#bug de Vmax?

data = [1.0, 2.0, 3.0]

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

In [5]:
Vmax, L, D = data

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

In [6]:
Vmax

1.0

Primeiro um exemplo do Julia
https://github.com/JuliaLang/julia/issues/21867

In [7]:
function foo()
  a = zeros(3)

  @sync begin
    for i = 2:nprocs
      println(i)
      @async begin
        a += fetch(@spawnat i (xxx = ones(3); xxx)) # BUG LINE
      end
    end
  end

  println(a)
end

foo()

2
3
4
[1.0, 1.0, 1.0]


In [8]:
function foo()
  a = zeros(3)

  @sync begin
    for i = 2:nprocs
      println(i)
      @async begin
        bbb = fetch(@spawnat i (xxx = ones(3); xxx))
        a += bbb
      end
    end
  end

  println(a)
end

foo()

2
3
4
[3.0, 3.0, 3.0]


In [9]:
#acho que consegui replicar o bug, "a" está aparecendo com uma variável global para todos os processadores
function foo()
  a = 1

  @sync begin
    for i = 2:nprocs
      println(i)
      @async begin
                
                for n in 1:10
                    for s in 1:3
                        a+=1
                    end
                end
                
                println(a)
      end
    end
  end

#   println(a)
end

foo()

#o valor de "a" está acumulando entre os processadores que começam o processo ao mesmo tempo

2
3
4
31
61
91


In [10]:
#acho que consegui replicar o bug, "a" está aparecendo com uma variável global para todos os processadores
function foo()
  

  @sync begin
    for i = 2:nprocs
      println(i)
      @async begin
                a = 1
                for n in 1:10
                    for s in 1:3
                        a+=1
                    end
                end
                
                println(a)
      end
    end
  end

#   println(a)
end

foo()

#o valor de "a" está acumulando entre os processadores que começam o processo ao mesmo tempo

2
3
4
31
31
31


In [11]:
#acho que consegui replicar o bug, "a" está aparecendo com uma variável global para todos os processadores
function foo()
#   a = 1

  @sync begin
    for i = 2:nprocs
      println(i)
      @async begin
                
                a = 1
                
                for s in 1:10
                    
                    a+=1
                end
                
                println(a)
      end
    end
  end

#   println(a)
end

foo()

#o valor de "a" está acumulando entre os processadores que começam o processo ao mesmo tempo

2
3
4
11
11
11


In [12]:
function foo()
  a = 1

  @sync begin
    for i = 2:nprocs
      println(i)
      @async begin
                
                for s in 1:10
                    
                    a+=1
                end
                
                println(a)
      end
    end
  end

#   println(a)
end

foo()


2
3
4
11
21
31


teste 1

In [13]:
@everywhere function simulate_game(U)
    
    draw = 1
    
    u = U[draw]
    draw+=1
    
    
    for i in 1:10
        choose_parameters!(U,draw)
    end
    
    println(draw)
end


@everywhere function choose_parameters!(U,draw)
    u = U[draw]
    draw+=1
    u = U[draw]
    draw+=1
    
end


U = rand(10);

In [15]:
function foo()
    
    for w in 2:nprocs
        println("worker = $w")
        
        @async begin
                
                for s in 1:3
                    simulate_game(U)
                end
                
                println(a)
          end
        
        
        
    end
    
end
        
    
foo()
    

worker = 2
worker = 3
worker = 4
2
2
2
2
2
2
2
2
2


In [16]:
function foo()
    
    for w in 2:nprocs
        println("worker = $w")
        
        @async begin
            
            @fetchfrom w simulate_game(U)
            
        end
        
    end
    
end
    
        
    
foo()
    

worker = 2
worker = 3
worker = 4


teste 2

In [17]:
@everywhere function simulate_game(U)
    
    draw = 1
    
    u = U[draw]
    draw+=1
    
    
    for i in 1:10
        draw = choose_parameters(U,draw)
    end
    
    println(draw)
end


@everywhere function choose_parameters(U,draw)
    u = U[draw]
    println(u)
    draw+=1
#     u = U[draw]
#     println(u)
#     draw+=1
    
    return draw
end


U = rand(3, 11);

      From worker 2:	2
      From worker 3:	2
      From worker 4:	2


In [18]:
U[1,:]

11-element Array{Float64,1}:
 0.5508259654237029
 0.7185225869550229
 0.8104421576479541
 0.3032269268505967
 0.5781970371879841
 0.7100475836613889
 0.6126839401885806
 0.6099283947118745
 0.16581959068789964
 0.9976796337432927
 0.17748227952648188

In [19]:
draws = rand(13)

13-element Array{Float64,1}:
 0.5326714670351869
 0.044038392050065545
 0.8063651646530217
 0.1912084243253136
 0.5970568773062206
 0.6362126273905242
 0.5855019540248767
 0.39416134473804565
 0.6486375627224248
 0.6672274597161365
 0.3272768359336198
 0.8124507986318981
 0.012959739989115926

In [20]:
function foo()
    
    
    @sync begin

        for w in 2:nprocs
            println("worker = $w")

            @async begin

                @fetchfrom w simulate_game(draws)

            end

        end
    end
    
end
    
        
    
foo()
    

worker = 2
worker = 3
worker = 4
      From worker 2:	0.044038392050065545
      From worker 2:	0.8063651646530217
      From worker 2:	0.1912084243253136
      From worker 2:	0.5970568773062206
      From worker 2:	0.6362126273905242
      From worker 2:	0.5855019540248767
      From worker 2:	0.39416134473804565
      From worker 2:	0.6486375627224248
      From worker 2:	0.6672274597161365
      From worker 2:	0.3272768359336198
      From worker 2:	12
      From worker 4:	0.044038392050065545
      From worker 4:	0.8063651646530217
      From worker 4:	0.1912084243253136
      From worker 4:	0.5970568773062206
      From worker 4:	0.6362126273905242
      From worker 4:	0.5855019540248767
      From worker 4:	0.39416134473804565
      From worker 4:	0.6486375627224248
      From worker 4:	0.6672274597161365
      From worker 4:	0.3272768359336198
      From worker 4:	12
      From worker 3:	0.044038392050065545
      From worker 3:	0.8063651646530217
      From worker 3:	0.19120842

teste3: investigando se é while

In [21]:
@everywhere function simulate_game(U, draw)
    
#     draw = 1
    
    u = U[draw]
    draw+=1
    
    for i in 1:10
        draw = choose_parameters(U,draw)
        println("draw = $draw")
    end
    
    
end


@everywhere function choose_parameters(U,draw)
    u = U[draw]
    println(u)
    draw+=1
    u = U[draw]
    println(u)
    draw+=1
    
    return draw
end


Draws = rand(3, 23);

In [22]:
Draws[2,:]

23-element Array{Float64,1}:
 0.16133924558710944
 0.720764540075433
 0.821414485438906
 0.8284933276781472
 0.5862560137288213
 0.36605701976156113
 0.4028384999778585
 0.3697239427269967
 0.01929240107819763
 0.7675855623578047
 0.2783728591335284
 0.027112824266222635
 0.7869295767457873
 0.17983619439263765
 0.5409301170887992
 0.31495737271717084
 0.90765220690012
 0.3069133150725398
 0.3336899271096214
 0.6413541003914953
 0.7704768758339746
 0.3524743858516288
 0.9263271688090875

In [23]:
result = zeros(5)

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [24]:
t = 10
T = 10

10

In [25]:
(result[1] ==0.0 && t < T)

false

In [26]:
using DelimitedFiles

writedlm( "Draws.csv",  Draws, ',')

In [27]:
Draws = readdlm("Draws.csv", ',')

3×23 Array{Float64,2}:
 0.746409  0.325699  0.484006  0.322119  …  0.418954  0.751789   0.684148
 0.161339  0.720765  0.821414  0.828493     0.770477  0.352474   0.926327
 0.300567  0.731411  0.194212  0.418401     0.12586   0.0265533  0.714992

In [28]:
function foo()
    @sync begin

        for w in 2:nprocs
            println("worker = $w")

            @async begin

                @fetchfrom w simulate_game(Draws[(w-1), :])

            end

        end
    end
end
    
        
    
foo()
    

worker = 2
worker = 3
worker = 4
      From worker 3:	0.720764540075433
      From worker 3:	0.821414485438906
      From worker 3:	0.8284933276781472
      From worker 3:	0.5862560137288213
      From worker 3:	0.36605701976156113
      From worker 3:	0.4028384999778585
      From worker 3:	0.3697239427269967
      From worker 3:	0.01929240107819763
      From worker 3:	0.7675855623578047
      From worker 3:	0.2783728591335284
      From worker 3:	0.027112824266222635
      From worker 3:	0.7869295767457873
      From worker 3:	0.17983619439263765
      From worker 3:	0.5409301170887992
      From worker 3:	0.31495737271717084
      From worker 3:	0.90765220690012
      From worker 3:	0.3069133150725398
      From worker 3:	0.3336899271096214
      From worker 3:	0.6413541003914953
      From worker 3:	0.7704768758339746
      From worker 3:	22
      From worker 4:	0.7314105949879297
      From worker 4:	0.1942118368133725
      From worker 4:	0.4184008388701921
      From worker 4:	

In [22]:
#trocando @async de lugar

Investigando se o problema é que tem uma função dentro da outra

In [81]:
a = [2, 3]

(a)' * a

13

In [43]:
FC = zeros(2)

pids = workers()
lengthworkers = Int64(length(pids)/2)
pool = pids[1:lengthworkers]

f = criterion

criterion (generic function with 1 method)

In [97]:
function fill_solutions!(pids, pool, FC, f, Draws, S, N, n1)
    
     
    @time begin

        @sync for (i, w) in enumerate(pool)
            @async FC[i] = @fetchfrom w compute_solutions(f, Draws, S, N, n1, pids[i + length(pool)])
        end
        
        
    end
    
end

fill_solutions! (generic function with 1 method)

In [98]:
@everywhere function compute_solutions(f, Draws, S, N, n1, aux)
    
    println("aux id = $aux")
    
    fc = f(Draws, S, N, n1)
    
    
    return fc
end

In [149]:
@everywhere function criterion(Draws, S, N, n1)
      


    function simulate_moments(Draws, S, N, n1)
        
        Results = loop_simulations(Draws, S, N, n1)
        
        mm = Results[1, 1, :]
        return mm
                
    end
    

    function loop_simulations(Draws, S, N, n1)
        
        Results = zeros(S, N, 2)
        
        T = 10
        t = 1
        draw = 1
        
        for s in 1:S
            for n in 1:N
                if(n <= n1)
                    Results[s,n,1:2] .= simulate_game(Draws[S, N, :], T, t, draw)
                else
                    Results[s,n,1:2] .= simulate_game(Draws[S, N, :], T, t, draw)
                end

            end
        end
        
        return Results

    end

    function simulate_game(Draws, T, t, draw)

        result = zeros(2)

        while(result[1] == 0.0 && t < T)


            u1 = Draws[draw]
            draw+=1

            u2 = Draws[draw]
            draw+=1

            u3 = Draws[draw]
            draw+=1

#             if(u1 < 0.5)
#                 result[1] = 1.0
#             end
            
            u2, u3 = choose_parameters(u2, u3)
            
            t +=1
        end
        
        return result
    end
            
    


    function choose_parameters(u2, u3)
        
        return u2, u3
        
    end
   
    
    
    simulated_moments = simulate_moments(Draws, S, N, n1)
    
    
    
    err = (simulated_moments)' * (simulated_moments)
    
    return err
    
    #end of criterion
end


In [150]:
S = 100
N = 66
n1 = 17

Draws = rand(S, N, 10 * 3);

In [193]:
@show FC
fill_solutions!(pids, pool, FC, f, Draws, S, N, n1)

FC = [0.0, 0.0]
      From worker 2:	aux id = 4
      From worker 3:	aux id = 5
  0.044819 seconds (289 allocations: 13.234 KiB)


Fim da investigação inicial de julia

In [4]:
@everywhere using DataFrames, LinearAlgebra, Statistics

In [ ]:
function loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)


    #U is an array if dimensions S x N x 120 of fixed draws of the uniform distribution
        #120 is the upper bound of draws needed: 4 for each round (with cram dowm), maximum of 30 rounds
    #S is the number of simulations
    #N is the number of observations

    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster




    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas


    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(U[s,n,:], data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(U[s,n,:], data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end

        end
    end

    return Results

end

In [ ]:
function simulate_game(U_sn, data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)


    #U_sn is a vector of U made for simulation s of observation n

    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj


    c0 = game_parameters[3];
    c1 = c0/30;

    ρ = game_parameters[1];
    λj = game_parameters[4];


    draw = 1 #index to indicate which draw it is



#     hst = Hs0
#     hjt = Hj0

    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt

    result = zeros(5);

    #number of observed proposals
    observed_proposals = 0.0


    #recovering "T+1" from s_W 
    T = size(s_W,1)

    println("T = $T")


    while(result[1]==0.0 && t < T)


        u = U_sn[draw]
        draw+=1

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end


        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next, draw = choose_parameters(U_sn, draw, propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end



        if(policy==3.0)

            observed_proposals += 1.0

            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)

            t+=1

            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



        else      
            #(policy==1.0)

            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)



            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)

            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)

        end





    end


    return result
end


In [ ]:
function choose_parameters(U_sn, draw, propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

    if(propositor=="s")

        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst

        hmt = hjt
        lmt = ljt


        lk_next = hkt


        u = U_sn[draw]
        draw+=1
        hk_next = draw_beta(u, hkt)

        u = U_sn[draw]
        draw+=1
        hm_next = draw_beta(u, hmt)

    else
#             propositor=="j"
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt

        hmt = hst
        lmt = lst


        lk_next = hkt


        u = U_sn[draw]
        draw+=1
        hk_next = draw_beta(u, hkt)

        u = U_sn[draw]
        draw+=1
        hm_next = draw_beta(u, hmt)
    end

    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next, draw
end

In [ ]:
@everywhere function criterion(U, S, N, data_moments, W_hat, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    #calculate the value of the objective function
    
    
    function simulate_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)


        Results = loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

        Moments = zeros(S, 8)

        μ = simulation_parameters[3]
        for s in 1:S

                DF = DataFrame()
                DF.payoff_s = Results[s,:,1];
                DF.payoff_j = Results[s,:,2];
                DF.out = Results[s,:,3];
                DF.t = Results[s,:,4];
                DF.observed_proposals = Results[s,:,5];
                DF.Ds = Results[s,:,6];
                DF.Dj = Results[s,:,7];


                DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


                #desfazendo o deslocamento no índice de t
                DF.t = DF.t .- 1.0;

                # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

                INCOURT = filter(DF -> DF.t .> 0.0, DF);
                PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

                PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
                INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

                #cálculo dos momentos
                #1. avg log number of months between observed proposals incourt

                mm1 = INCOURT
                mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
                mm1 = mean(mm1)

                #2. fraction reorganized given that the case went into court

                mm2 = INCOURT
                mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

                #3. ln duration of court cases in months

                mm3 = INCOURT
                mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
                mm3.t = mm3.t .* μ
                mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

                #4. fraction of cases incourt
                mm4 = size(INCOURT,1) / size(DF, 1)

                #5. avg recovery rate for senior given precourt REORGANIZATION

                #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

                mm5 = PRECOURT_REORG
                mm5.R_s = mm5.payoff_s ./ mm5.Ds
                mm5 = mean(mm5.R_s)



                #6. avg recovery rate for junior given precourt REORGANIZATION

                #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

                mm6 = PRECOURT_REORG
                mm6.R_j = mm6.payoff_j ./ mm6.Dj
                mm6 = mean(mm6.R_j)


                #7. junior avg fraction gain given incourt REORGANIZATION

                mm7 = INCOURT_REORG
                mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

                #8. total recovery rate given incourt REORGANIZATION

                mm8 = INCOURT_REORG
                mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)


                Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]


                #if there are no INCOURT or no PRECOURT cases, it will return NaN
                #thus, we replace it by zero
                Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

            end
        return vec(mean(Moments, dims=1))
    end
    

    function loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)


        #U is an array if dimensions S x N x 120 of fixed draws of the uniform distribution
            #120 is the upper bound of draws needed: 4 for each round (with cram dowm), maximum of 30 rounds
        #S is the number of simulations
        #N is the number of observations
        
        #data1 is the center of the first cluster
        #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
        #n1 is the number of observations in the first cluster




        #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
        Results = zeros(S, N, 7)

        #últimas entradas de Results são os valores das dívidas


        for s in 1:S
            for n in 1:N
                if(n <= n1)
                    Results[s,n,1:5] .= simulate_game(U[s,n,:], data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                    Results[s,n, end-1] = A1[5]
                    Results[s,n, end] = A1[6]
                else
                    Results[s,n,1:5] .= simulate_game(U[s,n,:], data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                    Results[s,n, end-1] = A2[5]
                    Results[s,n, end] = A2[6]
                end

            end
        end

        return Results

    end
    
    
    function rand_fixo!(U, draw)
        
        #U is a vector with fixed draws of the uniform distribution
        #draw is an index to indicate which draw it is
        #draw must be an unitary vector, ie, draw = [1], so it will be mutable
        
        
        u = U[draw]
    
        draw[1] += 1

        return u[1] #necessário para que o output seja uma variável, e não um array
    end
        
    function simulate_game(U_sn, data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)


        #U_sn is a vector of U made for simulation s of observation n

        Vmax = data[1]
        L = data[2]
        Dj = data[3]
        Ds = one(1.0) - Dj


        c0 = game_parameters[3];
        c1 = c0/30;

        ρ = game_parameters[1];
        λj = game_parameters[4];


        draw = 1 #index to indicate which draw it is



    #     hst = Hs0
    #     hjt = Hj0

        hst = Int64(simulation_parameters[1])
        hjt = Int64(simulation_parameters[2])

        #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
        lst = hst
        ljt = hjt

        result = zeros(5);

        #number of observed proposals
        observed_proposals = 0.0


        #recovering "T+1" from s_W 
        T = size(s_W,1)
        
        println("T = $T")


        while(result[1]==0.0 && t < T)


            u = U_sn[draw]
            draw+=1

            if(u < λj)
                propositor = "j"

            else
                propositor = "s"

            end


            #setting the default variables according to the propositor
            Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next, draw = choose_parameters(U_sn, draw, propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
            
            #proposal ####
            policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

            #lm_next is the update of the adversary's lower bound
            if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
                lm_next = lmt
            else
                lm_next = Int64(lm_next)
            end



            if(policy==3.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    t+=1

                    hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
                end

            elseif(policy==2.0)

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



            else      
                #(policy==1.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)



                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 1.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals


                end
            end


            if(t==T)

                result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
                result[2] = j_W[T, 1, 1, 1]
                result[3] = 1.0
                result[4] = t
                result[5] = max(observed_proposals, 1.0)

            end





        end


        return result
    end

    
    function choose_parameters(U_sn, draw, propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

        if(propositor=="s")

            Pkt_array = Pst_array
            Cont_val = j_W
            prop_index = 1
            respondent_index = 2 #índice de j, para organizar o payoff
            m_L = j_L

            hkt = hst
            lkt = lst

            hmt = hjt
            lmt = ljt


            lk_next = hkt


            u = U_sn[draw]
            draw+=1
            hk_next = draw_beta(u, hkt)

            u = U_sn[draw]
            draw+=1
            hm_next = draw_beta(u, hmt)

        else
#             propositor=="j"
            Pkt_array = Pjt_array
            Cont_val = s_W
            prop_index = 2
            respondent_index = 1
            m_L = s_L

            hkt = hjt
            lkt = ljt

            hmt = hst
            lmt = lst


            lk_next = hkt


            u = U_sn[draw]
            draw+=1
            hk_next = draw_beta(u, hkt)

            u = U_sn[draw]
            draw+=1
            hm_next = draw_beta(u, hmt)
        end

        return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next, draw
    end


    function draw_beta(u, hkt, β=game_parameters[2], grid=100)


        if(hkt == grid)

            return grid

        else

            x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
            x = round(x * grid, digits=0)
    #         return Int64(x * 100) #to convert in an integer
            return Int64(x)

        end
    end

    
    #cost function
    function Ct(t, c0=c0, c1=c1)

        #didn't use D because is normalized to D == 1.0
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 + c1 * (t-1) #test to make index==1 be t==0
        end
    end



    #liquidation payoffs
    function s_L(t, L, Ds, Dj, c0, c1)
        return min(L - Ct(t, c0, c1), Ds)
    end


    function j_L(t, L, Ds, Dj, c0, c1)
        return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
    end


    function proposal(Pkt_array, t, hkt, lmt)

        #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

        return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]

    end


    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax
        end


    end

    # answer_liq
    function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)

        liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)

        payoff_liq, answer = findmax(liq)

        return payoff_liq, answer
    end

    function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)

        reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

        payoff_reorg, answer = findmax(reorg_value)

        return payoff_reorg, answer
    end

    function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)

        if(propositor=="s")
            hst = hk_next
            lst = lk_next

            hjt = hm_next
            ljt = lm_next

        else
            hst = hm_next
            lst = lm_next

            hjt = hk_next
            ljt = lk_next
        end


        return hst, lst, hjt, ljt
    end

    
    #simulation####
    simulated_moments = simulate_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    err = error_vector(data_moments, simulated_moments)
    
    #* in this case represents matrix multiplication
    criterion_val = transpose(err) * W_hat
    criterion_val = criterion_val * err
    
#     return (transpose(err) * W_hat) * err
    return criterion_val
    
end

Simulated Annealing

In [ ]:
function fill_initial_solutions!(pids, pool, T, ug, lg, us, ls, FCvals, Gvals, Svals, g, s, f, U, S, N, data_moments, W_hat, data1, data2, n1)
    
    
    @time begin

        @sync for (i, w) in enumerate(pool)
            @async FCvals[i], Gvals[i, :], Svals[i, :] = @fetchfrom w compute_initial_solutions(pids, pids[i+length(pool)], T, ug, lg, us, ls, g, s, f, U, S, N, data_moments, W_hat, data1, data2, n1)
        end
        
        
    end
    
    
end


In [ ]:
@everywhere function compute_solutions(pids, aux, T, ug, lg, us, ls, fc, g, s, gamma, Tac, max_energy, f, U, S, N, data_moments, W_hat, data1, data2, n1)
    
    
    num_digits = 3;
    
    #we generate new gtest
    dg = draw_delta(T, size(g,1)) .* (ug .- lg)
    
    gtest = g .+ dg

    #next step is to keep solution within bounds
    gtest = round.(gtest, digits=num_digits)

    gtest = (gtest .< lg) .* lg .+ (lg .<= gtest) .* (gtest .<= ug) .* gtest .+ (ug .< gtest) .* ug
    

    #we evaluate the function and the change between the test point and the current point
    #for this we will have to solve the game, since we changed the game_parameters
    A2 = @spawnat aux solve_tree(data2, gtest)
    
    A1 = solve_tree(data1, gtest)
    
    A2 = fetch(A2)
    
    
#     A1 = @async @fetchfrom aux solve_tree(data1, gtest)
#     A2 = @sync solve_tree(data2, gtest)

    #now we start another loop to make changes in s, the simulation parameters
    for t2 in 1:(length(pids)*5) #the cost to test simulaton parameters is lower


        #we generate new stest points using mu_inv function
        ds = draw_delta(T, size(s,1)) .* (us .- ls)

        #we need to round because these are changes in θs0, θj0, which will be used as integers
        ds[1:2] .= round.(ds[1:2], digits=0)

        #ds[3] is μ, which can have 3 digits
        ds[3] = round(ds[3], digits=num_digits)

        stest = s .+ ds

        #next step is to keep solution within bounds
        #this is just a multiplication using indicator functions
        
        stest = round.(stest, digits=num_digits)
        
        stest = (stest .< ls) .* ls .+ (ls .<= stest) .* (stest .<= us) .* stest .+ (us .< stest) .* us
        
        
        
        
        #message so that we can see the upgrade
#         println("gtest = $gtest, stest = $stest")

        #now we evaluate the criterion function directly
        #we don't need to solve the model again because we didn't change the game parameters

        #evaluating the criterion function
        #gtest = game_parameters in this test
        #stest = simulation parameters in this test
        
        

        ftest = f(U, S, N, data_moments, W_hat, gtest, stest, data1, data2, A1, A2, n1)
        
        
        #accept if ftest < fc or if (ftest > fc and acceptance function says to accept)
        if ((ftest < fc || rand() < acceptance_function(gamma, fc, Tac, max_energy))==true)
            g = gtest
            s = stest
            fc = ftest 
       
        end
    end
    
    return fc, g, s
    
end

In [ ]:
function fill_solutions!(pids, pool, T, ug, lg, us, ls, FCvals, Gvals, Svals, gamma, Tac, max_energy, f, U, S, N, data_moments, W_hat, data1, data2, n1)
    #     FCvals = SharedArray{Float64}(length(pids))
    #     Gvals = SharedArray{Float64, 2}(length(pids), size(g,1))
    #     Svals = SharedArray{Float64, 2}(length(pids), size(s,1))
        #these values come from the master worker, no need to re-create arrays inside here
    
    
    #cada trabalhador vai trabalhar usando a sua solução anterior.
    @time begin

        @sync for (i, w) in enumerate(pool)
            @async FCvals[i], Gvals[i, :], Svals[i, :] = @fetchfrom w compute_solutions(pids, pids[i+length(pool)], T, ug, lg, us, ls, FCvals[i], Gvals[i, :], Svals[i, :], gamma, Tac, max_energy, f, U, S, N, data_moments, W_hat, data1, data2, n1)
        end
    
        
    end
    
end


In [ ]:
function compute_gamma(FCvals, max_energy, Tac)
    
    #FCvals = array with the values of the criterion function. Example: E(xi) for every i in 1, ..., m
    #max_energy = maximum energy from last round, calculated from FCvals
    #Tac = acceptance temperatura
    
    gamma = zero(0.0)
    
    for i in FCvals
        gamma += exp((i - max_energy)/Tac)
    end
    
    return gamma
end

    
    

In [ ]:
#analisando acceptance_function primeiro

@everywhere function acceptance_function(gamma, fc, Tac, max_energy)
    
    #gamma is the gamma calculated as eq(15) in Xavier et al. (2010) paper.
    #max_energy is the maximum value of the criterion function among all the workers]
    #fc is the current energy level, i.e., E(xi)
    #Tac is the acceptance temperature
    
    
    A = (fc - max_energy)/Tac
    A = exp(A)
    A = A/gamma
    
    return A
    
end




In [ ]:
function compute_var(m, gamma, FCvals, Tac, max_energy)
    
    #m is the number of workers, length(workers())
    
    #compute the variation in the acceptance probabilities
    
    A = zero(0.0)
    
    for fc in FCvals
        A += acceptance_function(gamma, fc, Tac, max_energy)^2
    end
    
    var = (1/m) * (A) - (1/m^2)
    return var
    
end

In [ ]:
function temp_max(tfinal, k, D)
    #k is the number of iterations
    #tfinal is the final temperature
    #D is the dimension of the vector of parameters
    
    #this function returns the initial temperature according to the cooling scheme proposed by Ingber(1989)
    #we use ci = c =  1
    #and Ti = T
    #that is, c and T are constant for all the parameters
    
    t0 = tfinal / ((exp(-k^(1/D))))
    
    return t0
    
    
end 

In [ ]:
function coupled_sim_anl(f, U, S, N, data1, data2, n1, data_moments, W_hat, g0, s0, lg, ug, ls, us, Kmax, T0, alpha, pids)
    
    #SA algorithm adapted from 
    #https://www.mathworks.com/matlabcentral/fileexchange/33109-simulated-annealing-optimization
    
    #inputs####
    #f : function to evaluate
    #S: number of simulations per observation
    #N: number of observations in the data set
    #data = [Vh/D, L/D, Dj/D]
    #W_hat: weighting matrix for criterion function
    #g0: initial game parameters
    #s0: initial simulation parameters
    #lg and ug: lower and upper bounds for game parameters
    #ls and us: lower and upper bounds for simulation parameters
    #Kmax: maximum number of temperatures
    #ntest: number of test points simulated for each temperature
    #TolFun: parameter of tolerance to help choose the appropriate solutions
    
    
    #T0 initial temperature
    
    
    #outputs####
    #the optimal values of the function and its parameters: f0, g0, s0
    #the best value of the function at each temperature and its parameters: fvals, gvals, svals
    
    @time begin



        #initializing the algorithm ####

        #calculating the initial energy levels
        g = g0
        s = s0
        D = size(g,1) + size(s,1) #size of the parameter vector
        
        
        lengthworkers = Int64(length(pids)/2)
        
        
        
        
        
        
        #arrays to store the energy level (function val), the g and s for each worker
#         FCvals = SharedArray{Float64}(lengthworkers)
#         Gvals = SharedArray{Float64, 2}(lengthworkers, size(g,1))
#         Svals = SharedArray{Float64, 2}(lengthworkers, size(s,1))
        
        
        FCvals = zeros(lengthworkers)
        Gvals = zeros(lengthworkers, size(g,1))
        Svals = zeros(lengthworkers, size(s,1))
        
        
        #pool of workers, it is half of the pids. 
        #Think of it as the pool of main workers, and the others the auxiliary workers
        pool = pids[1:lengthworkers]
        
        #initial definitions
        T = T0
        Tac = T0 #using the same initial temperature, after a few iterations it will be adjusted automatically
        
        
        #U is an S x N x 120 array with draws of the uniform distribution ####
        #it guarantees that the only variation from each simulation come from the change of the parameters
        
        
        fill_initial_solutions!(pids, pool, T, ug, lg, us, ls, FCvals, Gvals, Svals, g, s, f, U, S, N, data_moments, W_hat, data1, data2, n1)
    
        

        max_energy = findmax(FCvals[:])[1]
        gamma = compute_gamma(FCvals, max_energy, Tac)


        m = lengthworkers
        var_desired = 0.99 * ((m-1)/m^2)

        #para guardar os melhores valores de cada ciclo de temperatura
        fvals = zeros(Kmax)
        gvals = zeros(Kmax, size(g, 1))
        svals = zeros(Kmax, size(s, 1))


        #Main loop simulates de annealing Kmax times
        for k in 1:Kmax


            #calculating the new guesses
            fill_solutions!(pids, pool, T, ug, lg, us, ls, FCvals, Gvals, Svals, gamma, Tac, max_energy, f, U, S, N, data_moments, W_hat, data1, data2, n1)

            
            #show values for debugging
#             @show FCvals
#             @show Gvals
#             @show Svals
            
            
            #updates
            max_energy = findmax(FCvals[:])[1]
            gamma = compute_gamma(FCvals, max_energy, Tac)

            var = compute_var(m, gamma, FCvals, Tac, max_energy)

            if(var < var_desired)
                Tac = Tac * (1-alpha)
            else
                Tac = Tac * (1 + alpha)
            end


            #cools the system
            #cooling scheme based on Ingber 1989, as well as the draw_delta() function
            T = T0 * exp(-k^(1/D))


            #selecionando os melhores valores do teste
            fc, index = findmin(FCvals[:])
            g = Gvals[index, :]
            s = Svals[index, :]

            #record the value of the best solution of each cycle
            fvals[k] = fc
            gvals[k, :] .= g
            svals[k, :] .= s

            #message so that we can see the upgrade
            println(" end of cycle = $k, g = $g, s = $s, fc = $fc")



            end
        
         #end of @time
    end
    

    return fvals, gvals, svals
    
end
        
       


In [ ]:
f = criterion
S = 100
N = 66

#Vh/D, L/D, Dj/D
# data = [1.32, 0.32, 0.78]; 

#data_moments
data_moments = [2.11, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497];


W_hat = Matrix{Int}(I, 8, 8);


weights = [67.227, 485.003, 382.895, 2251.53, 60.4622, 32.3676, 1035.0, 1469.69] # bootstrap

W_hat = W_hat .* weights

In [ ]:
#ρ, β, c0, λj###
#bounds for g
lg = [0.4, 1.0, 0.01, 0.01]
ug = [0.93, 16.0, 0.18, 0.99]


g0 = lg .+ (ug .- lg)./2
#ρ was limited at 0.93 because a higher value would create more periods and calculations would take longer than we could afford.


#θs0, θj0, μ###
#bounds for s
#μ can be 0.5 month or 18 months
ls = [10, 10, 0.5] 
us = [90, 90, 36]


s0 = ls .+ (us .- ls)./2

Kmax = 2000

In [ ]:
alpha = 0.05
T0 = temp_max(0.001, Kmax, size(g0,1)+size(s0,1))

In [ ]:
#creating workers and assigning a seed to each one
pids = workers();

@everywhere using Random

startseed = 2020

for (i, w) in enumerate(pids)
    @spawnat w Random.seed!(startseed+i)
end


In [ ]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49 #number of observations in cluster1

data2 = [2.65, 0.69, 0.74];
n2 = 17 #number of observations in cluster2

In [ ]:
using DelimitedFiles

U = readdlm("U.csv", ',')
U = reshape(U, (S, N, 120));

In [ ]:
fvals, gvals, svals = coupled_sim_anl(f, U, S, N, data1, data2, n1, data_moments, W_hat, g0, s0, lg, ug, ls, us, Kmax, T0, alpha, pids);
@show fvals[Kmax]
@show gvals[Kmax, :]
@show svals[Kmax, :]

In [ ]:
using DelimitedFiles

writedlm( "fvals.csv",  fvals, ',')
writedlm("gvals.csv", gvals, ',')
writedlm("svals.csv", svals, ',')